In [1]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features,3)

In [3]:
model.load_state_dict(torch.load('best_model.pth'))

FileNotFoundError: [Errno 2] No such file or directory: 'best_model.pth'

In [ ]:
device = torch.device('cuda')
model = model.to(device)

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224,fps=5)
image = widgets.Image(format='jpeg', width=224, height=224)
red_slider = widgets.FloatSlider(description='red', min=0.0, max=1.0, orientation='vertical')
orange_slider = widgets.FloatSlider(description='orange', min=0.0, max=1.0, orientation='vertical')
green_slider = widgets.FloatSlider(description='green', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, red_slider,orange_slider,green_slider]))


In [ ]:
from jetbot import Robot
robot = Robot()

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    y = F.softmax(y, dim=1)
    
    prob_red = float(y.flatten()[2])
    prob_orange = float(y.flatten()[1])
    prob_green = float(y.flatten()[0])
    
    red_slider.value = prob_red
    orange_slider.value = prob_orange
    green_slider.value = prob_green
    
    if prob_red > 0.6:
        robot.forward(0.0)
        time.sleep(0.1)
        robot.stop()
        
    elif prob_orange >0.6:
        robot.backward(0.8)
        time.sleep(0.5)
        robot.stop()
        
    else prob_green >0.6:
        robot.forward(0.8)
        time.sleep(0.5)
        robot.stop()
        
    
        
update({'new': camera.value})  # we call the function once to intialize